## Linear Elasticity for Cantilever Beam

In [ ]:
using Gridap, Gridap.CellData, SparseArrays
using NearestNeighbors, Printf  # make sure this is at the top of your file
using Makie,GLMakie, LaTeXStrings, Makie.GeometryBasics

## 1. Problem
### Let's consider the 3D cantilever beam as follows:
<img src="imagines/3D_Cantilever.png" style="width:400px;height:250px"/>

The dimensions of beam are $L$ and $W$. 

This beam is fixed at $x=0$ and is applied a uniformly distributed load at $x=L$. The magnitude of the distributed load is $\mathbf{F} = [0, -1, 0]^T$. 

## Strong Form

**Equilibrium equation:**
$$
\nabla \cdot \boldsymbol{\sigma} + \mathbf{b} = \mathbf{0} \text{ in } \Omega,
$$

**Kinematics equation (strain–displacement relation):**
$$
\boldsymbol{\varepsilon} = \nabla_S \mathbf{u} = \frac{1}{2} \Big[ \nabla \mathbf{u} + (\nabla \mathbf{u})^T \Big].
$$

**Constitutive equation (stress–strain relation):**
$$
\boldsymbol{\sigma} = \mathbf{C} \boldsymbol{\varepsilon} 
\quad \text{or} \quad 
\boldsymbol{\sigma} = \lambda \, \text{tr}(\boldsymbol{\varepsilon})\, \mathbf{I} + 2\mu\, \boldsymbol{\varepsilon}.
$$

At the face with $x=L$ which is a boundary condition denoted by $\Gamma$, we have:

$$
\boldsymbol{\sigma} \cdot \mathbf{n} = \mathbf{F} = \begin{bmatrix}
0 & -1 & 0
\end{bmatrix}^T
 \text{ on } \Gamma,
$$

where $\boldsymbol{\sigma}$ is the stress tensor, $\mathbf{b}$ is the body force, $\mathbf{F}$ is the applied traction, and $\mathbf{n}$ is the outward unit normal.


## Derivation of the Weak Form from the Strong Form

### Step 1: Multiply by a Test Function
To derive the weak form, multiply the strong form by a test function $\mathbf{v} \in H^1(\Omega)$ (with $\mathbf{v} = \mathbf{0}$ on Dirichlet boundaries) and integrate over the domain $\Omega$:

$$
\int_{\Omega} (\nabla \cdot \boldsymbol{\sigma}) \cdot \mathbf{v} \, d\Omega = -\int_{\Omega} \mathbf{b} \cdot \mathbf{v} \, d\Omega.
$$
In this example, we assume that $\mathbf{b} = \mathbf{0}$, the we have:

$$
\int_{\Omega} (\nabla \cdot \boldsymbol{\sigma}) \cdot \mathbf{v} \, d\Omega = 0.
$$


### Step 2: Apply the Divergence Theorem
We have the equivalent form of $(\nabla \cdot \boldsymbol{\sigma}) \cdot \mathbf{v} $ as follows:

$$
(\nabla \cdot \boldsymbol{\sigma}) \cdot \mathbf{v} = \nabla \cdot (\boldsymbol{\sigma} \cdot \mathbf{v}) - \boldsymbol{\sigma} : \nabla \mathbf{v}.
$$
Then we have,
$$
\int_{\Omega} (\nabla_S^T \boldsymbol{\sigma}) \cdot \mathbf{v} \, d\Omega = \int_{\Omega} \left[ \nabla \cdot (\boldsymbol{\sigma} \cdot \mathbf{v}) - \boldsymbol{\sigma} : \nabla \mathbf{v} \right] \, d\Omega.
$$

Note that $\{\cdot\}$ is the dot product and $\{:\}$ is the inner product.

Apply the divergence theorem to $ \int_{\Omega} \nabla \cdot (\boldsymbol{\sigma} \cdot \mathbf{v}) \, d\Omega$:

$$
\int_{\Omega} \nabla \cdot (\boldsymbol{\sigma} \cdot \mathbf{v}) \, d\Omega = \int_{\partial \Omega} (\boldsymbol{\sigma} \cdot \mathbf{v}) \cdot \mathbf{n} \, d\Gamma = \int_{\partial \Omega} (\boldsymbol{\sigma} \cdot \mathbf{n}) \cdot \mathbf{v} \, d\Gamma.
$$

Because, we set the boundary condition $\boldsymbol{\sigma} \cdot \mathbf{n} = \mathbf{F}$ on $\Gamma$, then we have:

$$
\int_{\partial \Omega} (\boldsymbol{\sigma} \cdot \mathbf{n}) \cdot \mathbf{v} \, d\Gamma = \int_{\Gamma} \mathbf{F} \cdot \mathbf{v} \, d\Gamma.
$$

Thus:

$$
\int_{\Omega} (\nabla_S^T \boldsymbol{\sigma}) \cdot \mathbf{v} \, d\Omega = \int_{\Gamma} \mathbf{F} \cdot \mathbf{v} \, d\Gamma - \int_{\Omega} \boldsymbol{\sigma} : \nabla \mathbf{v} \, d\Omega = 0.
$$


#### Important notes
It should be noted that the very crucial step here is applying "DIVERGENCE THEOREM" to remove Gradient of Stress Tensor, there by reducing thef order of the partial differential equation. Readers interested in a deeper understanding are encourged to read Chapter 6 of the book: "Fish, Jacob, and Ted Belytschko. A first course in finite elements. Vol. 1. New York: Wiley, 2007.". Here is a screenshot of the "DIVERGENCE THEOREM" from Chapter 6 of that book.

<img src="imagines/divergence_theorem.png" />


### Step 3: Relation between Stress and Strain
The stress $\boldsymbol{\sigma}$ depends on the strain $\boldsymbol{\varepsilon}(\mathbf{u})$, where the strain tensor is:

$$
\boldsymbol{\varepsilon}(\mathbf{u}) = \frac{1}{2} \left( \nabla \mathbf{u} + (\nabla \mathbf{u})^T \right).
$$

Similarly, for the test function:

$$
\boldsymbol{\varepsilon}(\mathbf{v}) = \frac{1}{2} \left( \nabla \mathbf{v} + (\nabla \mathbf{v})^T \right).
$$

$$
\nabla \mathbf{v} = \boldsymbol{\varepsilon}(\mathbf{v}) + \frac{1}{2} \left( \nabla \mathbf{v} - (\nabla \mathbf{v})^T \right).
$$

$$\nabla \mathbf{v} = \boldsymbol{\varepsilon}(\mathbf{v}) + \boldsymbol{\omega}(\mathbf{v})$$

where $\boldsymbol{\omega}(\mathbf{v}) = \frac{1}{2} \left( \nabla \mathbf{v} - (\nabla \mathbf{v})^T \right)$, then we have:

$$
\boldsymbol{\sigma} : \nabla \mathbf{v} = \boldsymbol{\sigma} : \boldsymbol{\varepsilon}(\mathbf{v}) + \boldsymbol{\sigma} : \boldsymbol{\omega}(\mathbf{v}),
$$
because $\boldsymbol{\omega}(\mathbf{v}) $ is skew-symmetric and $\boldsymbol{\sigma}$ is symmetric, then $\boldsymbol{\sigma} : \boldsymbol{\omega}(\mathbf{v}) = 0$ (you can verify by yoursefl). Thus:

$$
\int_{\Omega} \boldsymbol{\sigma} : \nabla \mathbf{v} \, d\Omega = \int_{\Omega} \boldsymbol{\sigma} : \boldsymbol{\varepsilon}(\mathbf{v}) \, d\Omega.
$$


#### Julia code to verify that: $\mathbf{A} : \mathbf{B} = \text{tr}\left( \mathbf{A}^T \mathbf{B} \right)=0$. Where $\mathbf{A}$ is symmetric and $\mathbf{B}$ is  skew-symmetric.

In [ ]:
using LinearAlgebra
# Define the parameters
a1 = 1;b1 = 2;c1 = 3;d1 = 4;e1 = 5;f1 = 6;
# Build the symmetric matrix
A = [a1 d1 e1;
      d1 b1 f1;
      e1 f1 c1]
# Build the skew-symmetric matrix
B = [0 -a1 -b1;
      a1 0 -c1;
      b1 c1 0]
# Compute sum of dot product of B' and A
inner_product = tr(transpose(A) * B)
println(inner_product)


### Step 4: Formulate the Weak Form
Now, we have the weak form as follows:
$$
\int_{\Omega} \boldsymbol{\sigma}(\boldsymbol{\varepsilon}(\mathbf{u})) : \boldsymbol{\varepsilon}(\mathbf{v}) \, d\Omega = \int_{\Gamma} \mathbf{F} \cdot \mathbf{v} \, d\Gamma.
$$


### Final Weak Form to be used in Gridap.jl
Then, before using Gridap.jl, we define the bilinear form. Find $\mathbf{u}$ such that for all test functions $\mathbf{v}$:

$$
a(\mathbf{u}, \mathbf{v}) = l(\mathbf{v}),
$$

where:

$$
a(\mathbf{u}, \mathbf{v}) = \int_{\Omega} \big[ \boldsymbol{\sigma}(\boldsymbol{\varepsilon}(\mathbf{u})) : \boldsymbol{\varepsilon}(\mathbf{v}) \big] \, d\Omega,
\quad
l(\mathbf{v}) = \int_{\Gamma} \mathbf{F} \cdot \mathbf{v} \, d\Gamma.
$$

## Inputs

In [ ]:
L = 50 # Length
W = 10 # Width and Height
domain = (0, L, 0, W, 0, W) # sizes along X, Y, Z
partition = (50, 10, 10) # mesh sizes


### Add labels to left and right faces for boundary conditions

In [ ]:
model = CartesianDiscreteModel(domain, partition);
# Tag the left face (x = 0); right face (x = L)
labels = get_face_labeling(model)
add_tag_from_tags!(labels,"left", [1,3,5,7,13,15,17,19,25]); # left face: corners (1,3,5,7); edges (13,15,17,19); others (25) 
add_tag_from_tags!(labels, "right", [2,4,6,8,14,16,18,20,26]); # Right face: corners (2,4,6,8); edges (14,16,18,20); others (26) 
# writevtk(Ω,"Cantilever_Beam_3D")

### Construct weak form in Gridap.jl

In [ ]:
# Integrating on the domain Ω
degree = 2
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)
# neumann boundaries
Γ = BoundaryTriangulation(model,tags = "right")
dΓ = Measure(Γ,degree)
# Vector-valued FE space
order = 1
reffe = ReferenceFE(lagrangian, VectorValue{3,Float64}, order)
Vₕ = TestFESpace(Ω, reffe; conformity=:H1, dirichlet_tags = "left")
Uₕ = TrialFESpace(Vₕ)
# Constitutive law for LINEAR ELASTICITY 
const E = 119e3 # Young's modulus
const ν = 0.3   # Poisson's ratio
const λ = (E*ν)/((1+ν)*(1-2*ν)) # Lamé parameters
const μ = E/(2*(1+ν)) # Lamé parameters
ε₀(u) = 0.5 * ( ∇(u) + transpose(∇(u))) # Strain; In Gridap this can be automatically defined in Gridap.ε
σ(ε₀) = λ * tr(ε₀) * one(ε₀) + 2μ * ε₀; # Stress

## Run finite element analysis

In [ ]:
# The weak form
F(x) = VectorValue(0.0, -1, 0.0) # A unit load along Y axis applies to each node on the right face
a(u,v) = ∫((σ∘ε₀(u)) ⊙ ε₀(v)) *dΩ # Left-hand size; (∘) Composite functions
l(v) = ∫(F ⋅ v) * dΓ # Right-hand size
# Solution of the FE problem
op = AffineFEOperator(a,l,Uₕ,Vₕ)
uh = solve(op); # Solve the problem
# writevtk(Ω,"Cantilever_Beam_3D_uh",cellfields=["uh"=>uh,"epsi"=>ε₀(uh),"sigma"=>σ∘ε₀(uh)])


### Compute the compliance

In [ ]:
# Method 1
C1 = sum(∫((σ ∘ ε₀(uh)) ⊙ ε₀(uh)) * dΩ) # Compliance
println("Compliance 1: ", C1)
# Method 2
C2 = sum(∫(F ⋅ uh) * dΓ) # Compliance
println("Compliance 2: ", C2)
# Method 3
u = get_free_dof_values(uh) # displacement vector
K = get_matrix(op) # stiffness matrix
C3 = dot(u, K * u)
println("Compliance 3: ", C3)